# Los Simpsons con Big Data

## ETL

Implementación de la primera parte de la arquitectura. En la práctica se puede **prototipar** este ETL en un cuaderno de Jupyter para experimentar rápido y dejar algo funcional. Al finalizar, lo más adecuado es implementarlo en un script utilizando prácticas de **código de producción**.

### Extracción

Se desea automatizar la extracción de datos y no hacer un típico _copy and paste_, para lo cual se va a implementar un en un webscrapper que haga un crawl sobre las tablas del HTML.

En la aplicación se va a utilizar un _webscrapper_ que ya está implementado en Python dentro de funciones de la librería `pandas`

In [27]:
import os
import yaml
import pandas as pd
import numpy as np

In [35]:
with open("config.yml", "r") as file:
    config = yaml.safe_load(file); file.close()

PATH_GUESTS = config['scrapper']['url']
simpsons_raw = pd.read_html(PATH_GUESTS)
len(simpsons_raw)

23

### Transformación

In [40]:
seasons = np.arange(
    config['scrapper']['seasons'][0]
    ,config['scrapper']['seasons'][1]+1)

# for idx in seasons:
#     print(idx)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
